## Cleaning Data in Python

### 1. Common Data Problems

#### Numeric data or ... ?

In this exercise, and throughout this chapter, you'll be working with bicycle ride sharing data in San Francisco called ride_sharing. It contains information on the start and end stations, the trip duration, and some user information for a bike sharing service.

The user_type column contains information on whether a user is taking a free ride and takes on the following values:

* 1 for free riders.
* 2 for pay per ride.
* 3 for monthly subscribers.

In this instance, you will print the information of ride_sharing using .info() and see a firsthand example of how an incorrect data type can flaw your analysis of the dataset. The pandas package is imported as pd.

In [1]:
import pandas as pd

ride_sharing = pd.read_csv('ride_sharing_new.csv')

# Print the information of ride_sharing
print(ride_sharing.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25760 entries, 0 to 25759
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       25760 non-null  int64 
 1   duration         25760 non-null  object
 2   station_A_id     25760 non-null  int64 
 3   station_A_name   25760 non-null  object
 4   station_B_id     25760 non-null  int64 
 5   station_B_name   25760 non-null  object
 6   bike_id          25760 non-null  int64 
 7   user_type        25760 non-null  int64 
 8   user_birth_year  25760 non-null  int64 
 9   user_gender      25760 non-null  object
dtypes: int64(6), object(4)
memory usage: 2.0+ MB
None


In [2]:
# Print summary statistics of user_type column
print(ride_sharing['user_type'].describe())

# 3 values - 1,2,3 - maybe better to categorise this data

count    25760.000000
mean         2.008385
std          0.704541
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          3.000000
Name: user_type, dtype: float64


In [3]:
print(ride_sharing.head())

   Unnamed: 0    duration  station_A_id  \
0           0  12 minutes            81   
1           1  24 minutes             3   
2           2   8 minutes            67   
3           3   4 minutes            16   
4           4  11 minutes            22   

                                      station_A_name  station_B_id  \
0                                 Berry St at 4th St           323   
1       Powell St BART Station (Market St at 4th St)           118   
2  San Francisco Caltrain Station 2  (Townsend St...            23   
3                            Steuart St at Market St            28   
4                              Howard St at Beale St           350   

                    station_B_name  bike_id  user_type  user_birth_year  \
0               Broadway at Kearny     5480          2             1959   
1  Eureka Valley Recreation Center     5193          2             1965   
2    The Embarcadero at Steuart St     3652          3             1993   
3     The Embarcader

In [13]:
ride_sharing.columns

Index(['Unnamed: 0', 'duration', 'station_A_id', 'station_A_name',
       'station_B_id', 'station_B_name', 'bike_id', 'user_type',
       'user_birth_year', 'user_gender', 'user_type_cat', 'duration_trim',
       'duration_time'],
      dtype='object')

In [6]:
# Convert user_type from integer to category
ride_sharing['user_type_cat'] = ride_sharing['user_type'].astype('category')

# Write an assert statement confirming the change
assert ride_sharing['user_type_cat'].dtype == 'category'

# Print new summary statistics 
print(ride_sharing['user_type_cat'].describe())

count     25760
unique        3
top           2
freq      12972
Name: user_type_cat, dtype: int64


#### Summing strings and concatenating numbers

In the previous exercise, you were able to identify that category is the correct data type for user_type and convert it in order to extract relevant statistical summaries that shed light on the distribution of user_type.

Another common data type problem is importing what should be numerical values as strings, as mathematical operations such as summing and multiplication lead to string concatenation, not numerical outputs.

In [12]:
# Use the .strip() method to strip duration of "minutes" and store it in the duration_trim column.
ride_sharing['duration_trim'] = ride_sharing['duration'].str.strip('minutes')

# Convert duration_trim to int and store it in the duration_time column.
ride_sharing['duration_time'] = ride_sharing['duration_trim'].astype('int')

# Write an assert statement that checks if duration_time's data type is now an int.
assert ride_sharing['duration_time'].dtype == 'int'

# Print formed columns and calculate average ride duration 
print(ride_sharing[['duration','duration_trim','duration_time']])
print(ride_sharing['duration_time'].mean())

         duration duration_trim  duration_time
0      12 minutes           12              12
1      24 minutes           24              24
2       8 minutes            8               8
3       4 minutes            4               4
4      11 minutes           11              11
...           ...           ...            ...
25755  11 minutes           11              11
25756  10 minutes           10              10
25757  14 minutes           14              14
25758  14 minutes           14              14
25759  29 minutes           29              29

[25760 rows x 3 columns]
11.389052795031056


#### Tire size constraints

In this lesson, you're going to build on top of the work you've been doing with the ride_sharing DataFrame. You'll be working with the tire_sizes column which contains data on each bike's tire size.

Bicycle tire sizes could be either 26″, 27″ or 29″ and are here correctly stored as a categorical value. In an effort to cut maintenance costs, the ride sharing provider decided to set the maximum tire size to be 27″.

In this exercise, you will make sure the tire_sizes column has the correct range by first converting it to an integer, then setting and testing the new upper limit of 27″ for tire sizes.


In [68]:
# add tire column to end of df

ride_sharing['tire_sizes'] = np.random.randint(22,30, size=len(ride_sharing))

In [70]:
# Convert tire_sizes to integer
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')

# Set all values above 27 to 27
ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27

# Reconvert tire_sizes back to categorical
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')

# Print tire size description
print(ride_sharing['tire_sizes'].describe())

count     25760
unique        6
top          27
freq       9796
Name: tire_sizes, dtype: int64


#### Back to the future

A new update to the data pipeline feeding into the ride_sharing DataFrame has been updated to register each ride's date. This information is stored in the ride_date column of the type object, which represents strings in pandas.

A bug was discovered which was relaying rides taken today as taken next year. To fix this, you will find all instances of the ride_date column that occur anytime in the future, and set the maximum possible value of this column to today's date. Before doing so, you would need to convert ride_date to a datetime object.

The datetime package has been imported as dt, alongside all the packages you've been using till now.

In [71]:
# add rid_dt to df orignally as a object
import numpy as np

# Generate random dates and add them as a new column
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2025-01-01')

# Number of rows in the DataFrame
num_rows = ride_sharing.shape[0]

# Generate random dates within the specified range
random_dates = start_date + pd.to_timedelta(np.random.randint((end_date - start_date).days, size=num_rows), unit='D')

# Add the new column to the DataFrame
ride_sharing['ride_date'] = random_dates

# change to object data type as in slides
ride_sharing['ride_date'].astype('object')

0        2023-09-29 00:00:00
1        2017-09-27 00:00:00
2        2020-03-03 00:00:00
3        2017-09-16 00:00:00
4        2017-07-12 00:00:00
                ...         
25755    2018-02-27 00:00:00
25756    2019-03-09 00:00:00
25757    2023-11-14 00:00:00
25758    2019-01-17 00:00:00
25759    2020-07-06 00:00:00
Name: ride_date, Length: 25760, dtype: object

In [72]:
# import 
import datetime as dt
from datetime import datetime

# Convert ride_date to a datetime object using to_datetime(), then convert the datetime object into a date and store it in ride_dt column.
ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_date']).dt.date

# Save today's date
today = dt.date.today() #which is todays date

# For all instances of ride_dt in the future, set them to today's date.
ride_sharing.loc[ride_sharing['ride_dt'] > today, 'ride_dt'] = today

# Print the maximum date in the ride_dt column.
print(ride_sharing['ride_dt'].max())

2024-03-12


#### Finding duplicates

A new update to the data pipeline feeding into ride_sharing has added the ride_id column, which represents a unique identifier for each ride.

The update however coincided with radically shorter average ride duration times and irregular user birth dates set in the future. Most importantly, the number of rides taken has increased by 20% overnight, leading you to think there might be both complete and incomplete duplicates in the ride_sharing DataFrame.

In [74]:
#add ride_id to df
ride_sharing['ride_id'] = np.random.randint(1,99, size=len(ride_sharing))

# Find duplicated rows of ride_id in the ride_sharing DataFrame while setting keep to False.
duplicates = ride_sharing.duplicated(subset = 'ride_id', keep = False)

# Subset ride_sharing on duplicates and sort by ride_id and assign the results to duplicated_rides.
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')

# Print the ride_id, duration and user_birth_year columns of duplicated_rides in that order.
print(duplicated_rides[['ride_id','duration','user_birth_year']])

       ride_id    duration  user_birth_year
20904        1   6 minutes             1986
8951         1  13 minutes             1989
21067        1   7 minutes             1985
8937         1   8 minutes             1985
21108        1   9 minutes             1987
...        ...         ...              ...
4971        98  13 minutes             1996
13524       98  17 minutes             1985
13518       98   3 minutes             1981
14075       98   2 minutes             1991
15398       98  19 minutes             1972

[25760 rows x 3 columns]


#### Treating duplicates
In the last exercise, you were able to verify that the new update feeding into ride_sharing contains a bug generating both complete and incomplete duplicated rows for some values of the ride_id column, with occasional discrepant values for the user_birth_year and duration columns.

In this exercise, you will be treating those duplicated rows by first dropping complete duplicates, and then merging the incomplete duplicate rows into one while keeping the average duration, and the minimum user_birth_year for each set of incomplete duplicate rows.

In [75]:


# Drop complete duplicates in ride_sharing and store the results in ride_dup.
ride_dup = ride_sharing.drop_duplicates()

# Create the statistics dictionary which holds minimum aggregation for user_birth_year and mean aggregation for duration
statistics = {'user_birth_year': 'min', 'duration': 'mean'}

# Group by ride_id and compute new statistics
ride_unique = ride_dup.groupby(by = 'ride_id').agg(statistics).reset_index()

# Drop incomplete duplicates by grouping by ride_id and applying the aggregation in statistics.
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# Assert duplicates are processed
assert duplicated_rides.shape[0] == 0

TypeError: agg function failed [how->mean,dtype->object]

In [76]:
print(ride_sharing['ride_id'].describe())

count    25760.000000
mean        49.221467
std         28.204234
min          1.000000
25%         25.000000
50%         49.000000
75%         74.000000
max         98.000000
Name: ride_id, dtype: float64


### 2. Text and Categorical Data Problems

#### Finding consistency

In this exercise and throughout this chapter, you'll be working with the airlines DataFrame which contains survey responses on the San Francisco Airport from airline customers.

The DataFrame contains flight metadata such as the airline, the destination, waiting times as well as answers to key questions regarding cleanliness, safety, and satisfaction. Another DataFrame named categories was created, containing all correct possible values for the survey columns.

In this exercise, you will use both of these DataFrames to find survey answers with inconsistent values, and drop them, effectively performing an outer and inner join on both these DataFrames as seen in the video exercise. The pandas package has been imported as pd, and the airlines and categories DataFrames are in your environment.

In [102]:
airlines = pd.read_csv('airlines_final.csv')
airlines.columns


Index(['Unnamed: 0', 'id', 'day', 'airline', 'destination', 'dest_region',
       'dest_size', 'boarding_area', 'dept_time', 'wait_min', 'cleanliness',
       'safety', 'satisfaction', 'full_name'],
      dtype='object')

In [103]:
# defining categories
# use dict 
categories = {
    'cleanliness' : ['Clean', 'Average', 'Somewhat clean', 'Somewhat dirty', 'Dirty'],
    'safety' : ['Neutral' , 'Very safe', 'Somewhat safe', 'Very unsafe', 'Somewhat unsafe'],
    'satisfaction' : ['Very satisfied', 'Neutral', 'Somewhat satisfied', 'Somewhat unsatisfied', 'Very unsatisfied']
}
print(categories)

{'cleanliness': ['Clean', 'Average', 'Somewhat clean', 'Somewhat dirty', 'Dirty'], 'safety': ['Neutral', 'Very safe', 'Somewhat safe', 'Very unsafe', 'Somewhat unsafe'], 'satisfaction': ['Very satisfied', 'Neutral', 'Somewhat satisfied', 'Somewhat unsatisfied', 'Very unsatisfied']}


In [104]:
# Print categories DataFrame
print(categories)

# Print unique values of survey columns in airlines
print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
print('Safety: ',  airlines['safety'].unique(), "\n")
print('Satisfaction: ',  airlines['satisfaction'].unique(), "\n")

{'cleanliness': ['Clean', 'Average', 'Somewhat clean', 'Somewhat dirty', 'Dirty'], 'safety': ['Neutral', 'Very safe', 'Somewhat safe', 'Very unsafe', 'Somewhat unsafe'], 'satisfaction': ['Very satisfied', 'Neutral', 'Somewhat satisfied', 'Somewhat unsatisfied', 'Very unsatisfied']}
Cleanliness:  ['Clean' 'Average' 'Somewhat clean' 'Somewhat dirty' 'Dirty'] 

Safety:  ['Neutral' 'Very safe' 'Somewhat safe' 'Very unsafe' 'Somewhat unsafe'] 

Satisfaction:  ['Very satisfied' 'Neutral' 'Somewhat satsified' 'Somewhat unsatisfied'
 'Very unsatisfied'] 



In [121]:
# Create a set out of the cleanliness column in airlines using set() and find the inconsistent category by finding the difference in the cleanliness column of categories.
cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])

# Find rows of airlines with a cleanliness value not in categories and print the output.
cat_clean_rows = airlines['cleanliness'].isin(cat_clean)

# Print rows with inconsistent category
print(airlines[cat_clean_rows])

# Print rows with consistent categories only
print(airlines[~cat_clean])

Empty DataFrame
Columns: [Unnamed: 0, id, day, airline, destination, dest_region, dest_size, boarding_area, dept_time, wait_min, cleanliness, safety, satisfaction, full_name, wait_type, day_week]
Index: []


TypeError: bad operand type for unary ~: 'set'

#### Inconsistent categories

In this exercise, you'll be revisiting the airlines DataFrame from the previous lesson.

As a reminder, the DataFrame contains flight metadata such as the airline, the destination, waiting times as well as answers to key questions regarding cleanliness, safety, and satisfaction on the San Francisco Airport.

In this exercise, you will examine two categorical columns from this DataFrame, dest_region and dest_size respectively, assess how to address them and make sure that they are cleaned and ready for analysis. 

In [105]:
# Print the unique values in dest_region and dest_size respectively.
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']
['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium' 'Small     ' '    Large' 'Large     ']


From looking at the output, what do you think is the problem with these columns?

Answer - The dest_region column has inconsistent values due to capitalization and has one value that needs to be remapped. and The dest_size column has only inconsistent values due to leading and trailing spaces.

In [106]:
# Lower dest_region column and then replace "eur" with "europe"
airlines['dest_region'] = airlines['dest_region'].str.lower()
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

# Remove white spaces from `dest_size`
airlines['dest_size'] = airlines['dest_size'].str.strip()

# Verify changes have been effected
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand']
['Hub' 'Small' 'Medium' 'Large']


#### Remapping categories

To better understand survey respondents from airlines, you want to find out if there is a relationship between certain responses and the day of the week and wait time at the gate.

The airlines DataFrame contains the day and wait_min columns, which are categorical and numerical respectively. The day column contains the exact day a flight took place, and wait_min contains the amount of minutes it took travelers to wait at the gate. To make your analysis easier, you want to create two new categorical variables:

* wait_type: 'short' for 0-60 min, 'medium' for 60-180 and long for 180+
* day_week: 'weekday' if day is in the weekday, 'weekend' if day is in the weekend.

In [107]:
# Create the ranges and labels for the wait_type column mentioned in the description.
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create the wait_type column by from wait_min by using pd.cut(), while inputting label_ranges and label_names in the correct arguments.
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                                labels = label_names)

# Create the mapping dictionary mapping weekdays to 'weekday' and weekend days to 'weekend'.
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

# Create the day_week column by using .replace()
airlines['day_week'] = airlines['day'].replace(mappings)

#### Removing titles and taking names

While collecting survey respondent metadata in the airlines DataFrame, the full name of respondents was saved in the full_name column. However upon closer inspection, you found that a lot of the different names are prefixed by honorifics such as "Dr.", "Mr.", "Ms." and "Miss".

Your ultimate objective is to create two new columns named first_name and last_name, containing the first and last names of respondents respectively. Before doing so however, you need to remove honorifics.

In [111]:
# Replace "Dr." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Dr.","")

# Replace "Mr." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Mr.","")


# Replace "Miss" with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Miss","")

# Replace "Ms." with empty string ""
airlines['full_name'] = airlines['full_name'].str.replace("Ms.","")


#### Keeping it descriptive

To further understand travelers' experiences in the San Francisco Airport, the quality assurance department sent out a qualitative questionnaire to all travelers who gave the airport the worst score on all possible categories. The objective behind this questionnaire is to identify common patterns in what travelers are saying about the airport.

Their response is stored in the survey_response column. Upon a closer look, you realized a few of the answers gave the shortest possible character amount without much substance. In this exercise, you will isolate the responses with a character count higher than 40 , and make sure your new DataFrame contains responses with 40 characters or more using an assert statement.

In [118]:
# Store length of each row in survey_response column
resp_length = airlines['survey_response'].str.len()

# Find rows in airlines where resp_length > 40
airlines_survey = airlines[resp_length > 40]

# Assert minimum survey_response length is > 40
assert airlines_survey['survey_response'].str.len().min() > 40

# Print new survey_response column
print(airlines_survey['survey_response'])

KeyError: 'survey_response'

### 3. Advanced Data Problems

### 4. Record Linkage